<a href="https://colab.research.google.com/github/AchintyaX/Unsupervised_Sentiment_Analysis/blob/master/Hindi_newApproach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd 
import numpy as np
import nltk 
import re 
from re import sub
import string 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import sys
sys.path.append('/content/drive/My Drive/jio_project/sentiment_analysis/sentiment_hi')
import spacy
from spacy.lang.hi import Hindi

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors

In [0]:
hindi = pd.read_csv('/content/drive/My Drive/jio_project/sentiment_analysis/sentiment_hi/Hindi.csv')
hindi = hindi.dropna()
hindi = hindi.reset_index(drop=True)
hindi

,full_text,headline,language
0,छपरा। बिहार में सारण जिले के एकमा थाना क्षेत्र...,छपरा में व्यवसायी का शव बरामद,Hindi
1,We use cookies to ensure that we give you the ...,"गाजियाबाद: कांग्रेस नेता पूजा चड्ढा गिरफ्तार, ...",Hindi
2,We use cookies to ensure that we give you the ...,लाहौर में मारा गया खालिस्‍तान का टॉप लीडर Happ...,Hindi
3,अदनान सामी को पद्म श्री अवार्ड मिलने के बाद से...,पद्म श्री को लेकर शेरगिल-अदनान सामी में छिड़ा ...,Hindi
4,"First Published 28, Jan 2020, 10:30 AM IST\n\n...","मां, पत्नी और भतीजे से मिलते ही आधे घंटे तक फू...",Hindi
...,...,...,...
989,कुछ समय पहले सभी टेलीकॉम कंपनियों ने टैरिफ प्ल...,"जियो के ₹10 वाले रिचार्ज ने मचाई बड़ी खलबली, अ...",Hindi
990,"मेरे प्यारे दोस्तों और फॉलोअर्स को नमस्कार, एक...",41 साल की उम्र में भी यह अभिनेत्री बेहद हॉट और...,Hindi
991,"मेरे प्यारे दोस्तों और फॉलोअर्स को नमस्कार, एक...",ये अभिनेत्री अपनी अदाओं से सोशल मीडिया पर बना ...,Hindi
992,"मेरे प्यारे दोस्तों और फॉलोअर्स को नमस्कार, एक...",नीले कलर की साड़ी में यह एक्ट्रेस बेहद हॉट और ...,Hindi


In [0]:
news  = hindi['full_text'] # isolating the news articles
news = news.to_list() # converting it into an array 

len(news)

994

In [0]:
def preprocess_hin(article):
    text = sub(r'[a-zA-Z]','', article) 
    text = sub(r'[0-9][0-9]', '', text)
    text = sub(r'\n', '', text)
    text = sub(r',','', text)
    text = sub(r':','', text)
    text = sub(r'[0-9]', '', text)
    text = sub(r'।', '', text)
    text = sub(r'-','', text)
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    # text = emoji_pattern.sub(r'', text)
    text = text.strip(string.punctuation)

    return text 

In [0]:
def tokenize_hin(article):
    nlp = Hindi()
    doc = nlp(article)
    tokens = [token.text for token in doc]
    
    return tokens 

In [0]:
#  getting all the tokens 
for i in range(len(news)):
    news[i] = preprocess_hin(news[i])
    news[i] = tokenize_hin(news[i])
news[100]

['लखनऊ',
 ' ',
 '.',
 'बसपा',
 'सुप्रीमो',
 'मायावती',
 'ने',
 'नागरिकता',
 'संशोधन',
 'कानून',
 '(',
 'सीएए',
 ')',
 'और',
 'राष्ट्रीय',
 'नागरिकता',
 'रजिस्टर',
 '(',
 'एनआरसी',
 ')',
 'के',
 'विरोध',
 'में',
 'संघर्ष',
 'करने',
 'वाली',
 'महिलाओं',
 'के',
 'खिलाफ',
 'दर्ज',
 'मामले',
 'वापस',
 'लेने',
 'की',
 'मांग',
 'की',
 'है',
 '.',
 'पुलिस',
 'ने',
 'इस',
 'मामले',
 'में',
 'सौ',
 'से',
 'अधिक',
 'ज्ञातअज्ञात',
 'लोगों',
 'के',
 'खिलाफ',
 'मामले',
 'दर्ज',
 'कर',
 'आठ',
 'लोगों',
 'को',
 'गिरफ्तार',
 'किया',
 'है',
 '.',
 'मायावती',
 'ने',
 'सोमवार',
 'को',
 'ट्वीट',
 'किया',
 'सीएए',
 'एनआरसी',
 'आदि',
 'के',
 'विरोध',
 'में',
 'संघर्ष',
 'करने',
 'वाली',
 'महिलाओं',
 'समेत',
 'जिन',
 'लोगों',
 'के',
 'भी',
 'खिलाफ',
 'उप्र',
 'की',
 'भाजपा',
 'सरकार',
 'द्वारा',
 'गलत',
 'आधार',
 'पर',
 'मुकदमे',
 'दर्ज',
 'किए',
 'गए',
 'हैं',
 'उन्हें',
 'तुरन्त',
 'वापस',
 'लिया',
 'जाए',
 '.',
 'उन्होंने',
 'कहा',
 'सीएए',
 'और',
 'एनआरसी',
 'के',
 'विरोध',
 'में',
 'संघर्ष',
 'के',
 'दौर

In [0]:
# making a global dictionary
global_dict = []
for i in range(len(news)):
    for word in news[i]:
        global_dict.append(word)
len(global_dict)

367570

In [0]:
# finding all the unique words 
global_dict = list(set(global_dict))
len(global_dict)

17309

In [0]:
! pip3 install googletrans

In [0]:
import googletrans
from googletrans import Translator

In [0]:
translator = Translator()
print('वापस')
translation = translator.translate('वापस').text
print(translation)

वापस


JSONDecodeError: ignored

In [0]:
from textblob import TextBlob
blob = TextBlob('bad')
blob.polarity

-0.6999999999999998

In [0]:
# global_dict = global_dict.pop()
global_dict[990]

IndexError: ignored

In [0]:
word_sentiment_map = []
for i in global_dict:
    sentiment = {}
    translator = Translator()
    sentiment['word'] = i
    translation = translator.translate(i).text
    seniment['translation'] = translation
    sentiment['sentiment_value'] = TextBlob(translation).polarity

    word_sentiment_map.append(sentiment)
    print(sentiment)

JSONDecodeError: ignored

In [0]:
word_sentiment_map = []
exceptions = []
for i in global_dict:
    try:
        sentiment = {}
        sentiment['word'] = i
        translation = translator.translate(i).text
        sentiment['translation'] = translation
        sentiment['sentiment_value'] = TextBlob(translation).polarity 
        word_sentiment_map.append(sentiment)
        print(sentiment)
    except :
        exceptions.append(i)
len(exeptions)